In [46]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForSeq2SeqLM, AutoConfig, pipeline
from scipy.special import softmax
import pandas as pd
import re
import string
import torch
from sklearn import set_config

In [2]:
roberta_tokenizer = \
        AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
roberta_model = \
        AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation,
    remove words containing numbers, remove numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = ''.join([i for i in text if not i.isdigit()])
    return text

In [13]:
df = pd.read_csv("../raw_data/tweets.csv")
df.rename(columns = {'2022-05-31 15:38:40.948508':'date', "@Apple Pricey. That's iPhone.": "text" }, inplace = True)
df["text"] = df["text"].apply(clean_text)
X_pred = df["text"].tolist()

In [30]:
inputs = roberta_tokenizer(X_pred[0], return_tensors="pt")

with torch.no_grad():
    logits = roberta_model(**inputs).logits

predicted_class_id = logits.argmax().item()
roberta_model.config.id2label[predicted_class_id]

'Neutral'

In [23]:
%%time

prediction = []

for i in range(1000):
    inputs = roberta_tokenizer(X_pred[i], return_tensors="pt")

    with torch.no_grad():
        logits = roberta_model(**inputs).logits

    predicted_class_id = logits.argmax().item()
    prediction.append(roberta_model.config.id2label[predicted_class_id])

prediction

CPU times: user 3min 16s, sys: 5.4 s, total: 3min 21s
Wall time: 3min 45s


['Neutral',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Neutral',
 'Negative',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Neutral',
 'Negative',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Neutral',
 'Negative',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Neutral',
 'Negative',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Neutral',
 'Negative',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Neutral',
 'Negative',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Neutral',
 'Negative',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Neutral',
 'Negative',
 'Neutral',
 'Positive',
 'Ne

In [24]:
len(prediction)

1000

In [26]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

2022-06-01 11:42:53.314862: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [35]:
X_pred[4]

'end of thread  if you enjoyed this story make sure to rt and follow mbitpodcast lets see if we can get this in the hands of ericsyuan 🚀  cc zoom webex cisco microsoft apple emergencecap yahoobusiness business'

In [32]:
result = classifier(X_pred[4])
result

[{'label': 'NEGATIVE', 'score': 0.9881255626678467}]

In [42]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
encoded_input = tokenizer(X_pred[0], return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
scores

array([0.24754387, 0.6732406 , 0.07921556], dtype=float32)

In [52]:
roberta_model.

<bound method Module.parameters of RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=76

In [34]:
classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
